In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import unicodedata
from selenium import webdriver

from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.chrome.service import Service



import zipfile
from io import BytesIO
from datetime import datetime
from django.http import HttpResponse

import shutil
import os

In [2]:
#Scraping NBC news wild fires with Selenium and Beautifulsoup
def get_list_link_from_subject(subject, num_pages):


    PATH = "./chromedriver_win32/chromedriver.exe"

    s=Service(PATH)
    driver = webdriver.Chrome(service=s)

    link_list = []

    driver.get("https://www.google.com/search?q="+subject+"&rlz=1C1CHBF_frFR863FR863&biw=1920&bih=880&sxsrf=AOaemvI0XcPZB9YWw9GUVGwWTEXPDVqRxQ:1638967714934&source=lnms&tbm=nws&sa=X&ved=2ahUKEwjGlsnDntT0AhWTTcAKHeyuDk4Q_AUoAXoECAEQAw")

    driver.find_element(By.XPATH, "//button[@class='VfPpkd-LgbsSe VfPpkd-LgbsSe-OWXEXe-k8QpJ VfPpkd-LgbsSe-OWXEXe-dgl2Hf nCP5yc AjY5Oe DuMIQc']").click() #accept google policy

    for i in range(num_pages):
        if i != 0:
            driver.find_element(By.ID, "pnnext").click()

        html_source = driver.page_source

        soup = BeautifulSoup(html_source, 'lxml')

        #Getting all g-card 
        g_card_list = soup.find_all("g-card")

        for g_card in g_card_list:
            a = g_card.find("a")
            link = a['href']
            link_list.append(link)

    driver.quit()

    print("Successfully scraped : ", len(link_list), " links")

    return link_list



In [3]:
#Now that we have the link list, for each link try to scrape the article if there is one and the date if there is one.
def get_df_from_link_list(link_list):

    my_timeout = 10

    data = []

    for i, link in enumerate(link_list):
        d = {}

        try:
            html_text = requests.get(link, timeout=my_timeout).text

            soup = BeautifulSoup(html_text, 'lxml')

            title = soup.find('title')
            if title != None:
                d["Title"] = title.text

            d["Link"] = link
   
            time = soup.find('time')
            if time != None:
                if time.has_attr('datetime'):

                    d["Date"] = time['datetime']
            
            article = soup.find('article')
            if article != None:
                paragraphs = article.find_all('p')
                big_p = ""
                for p in paragraphs:
                    big_p = big_p + p.text + " "
                
                if big_p != "":
                    d["Content"] = unicodedata.normalize("NFKD", big_p).rstrip()
        except (requests.exceptions.Timeout, requests.exceptions.ConnectionError) as err: #Requests takes way too long
            print('Server taking too long. Try again later for page number' + str(i))

        data.append(d)

    return pd.DataFrame(data)

In [4]:
def get_df_from_subject(subject,num_pages):
    return get_df_from_link_list(get_list_link_from_subject(subject, num_pages))

In [5]:
def get_rid_of_date(df):
    res = df.drop(columns = ["Date"])
    return res

In [6]:
def get_article_list_from_subject(subject,num_pages):
    df = get_df_from_subject(subject,num_pages)
    articles = df["Content"]
    articles = articles.dropna()
    return list(articles)

In [7]:
def zip_list_of_articles(article_list, zip_file_name):
    os.mkdir("temp")
    for i,lines in enumerate(article_list):
        with open("temp/article_number_" + str(i) + ".txt",'w',encoding='utf-8') as f:
            f.write(lines)
    shutil.make_archive(zip_file_name, 'zip', "temp")
    shutil.rmtree("temp")

In [8]:
def zip_articles_from_subject(subject,num_pages,zip_file_name):
    article_list = get_article_list_from_subject(subject,num_pages)
    zip_list_of_articles(article_list,zip_file_name)

In [9]:
zip_articles_from_subject("Wildfire",10,"Wildfire_test")

Successfully scraped :  100  links
Server taking too long. Try again later for page number12
Server taking too long. Try again later for page number45
